Importing libraries

In [1]:
from pulp import *
import pandas

Reading and storing the data from diet_large.xlsx to a pandas dataframe

In [2]:
data = pandas.read_excel('diet_large.xls', nrows = 7146, header = 1, skip_row = 0)

Cleaning the data as it contains a lot of NaN values

In [3]:
data = data.set_index('Long_Desc')
data = data.fillna(value = 0)
food_data = data.iloc[:,:28]

In [4]:
food_data

,Protein,"Carbohydrate, by difference",Energy,Water,Energy.1,"Calcium, Ca","Iron, Fe","Magnesium, Mg","Phosphorus, P","Potassium, K",...,"Vitamin C, total ascorbic acid",Thiamin,Riboflavin,Niacin,Pantothenic acid,Vitamin B-6,"Folate, total",Vitamin B-12,Vitamin K (phylloquinone),Cholesterol
Long_Desc,,,,,,,,,,,,,,,,,,,,,
"Butter, salted",0.85,0.06,717,15.87,3000,24.0,0.02,2.0,24.0,24.0,...,0.0,0.005,0.034,0.042,0.110,0.003,3.0,0.17,7.0,215.0
"Butter, whipped, with salt",0.85,0.06,717,15.87,2999,24.0,0.16,2.0,23.0,26.0,...,0.0,0.005,0.034,0.042,0.110,0.003,3.0,0.13,7.0,219.0
"Butter oil, anhydrous",0.28,0.00,876,0.24,3665,4.0,0.00,0.0,3.0,5.0,...,0.0,0.001,0.005,0.003,0.010,0.001,0.0,0.01,8.6,256.0
"Cheese, blue",21.40,2.34,353,42.41,1477,528.0,0.31,23.0,387.0,256.0,...,0.0,0.029,0.382,1.016,1.729,0.166,36.0,1.22,2.4,75.0
"Cheese, brick",23.24,2.79,371,41.11,1552,674.0,0.43,24.0,451.0,136.0,...,0.0,0.014,0.351,0.118,0.288,0.065,20.0,1.26,2.5,94.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Frog legs, raw",16.40,0.00,73,81.90,305,18.0,1.50,20.0,147.0,285.0,...,0.0,0.140,0.250,1.200,0.000,0.120,15.0,0.40,0.1,50.0
"Fish, mackerel, salted",18.50,0.00,305,43.00,1276,66.0,1.40,60.0,254.0,520.0,...,0.0,0.020,0.190,3.300,0.000,0.410,15.0,12.00,7.8,95.0
"Mollusks, scallop, (bay and sea), cooked, steamed",23.20,0.00,112,73.10,469,115.0,3.00,55.0,338.0,476.0,...,0.0,0.100,0.060,1.300,0.000,0.100,12.0,1.30,0.2,53.0


Reading and storing constraints as pandas dataframes

In [5]:
constraints_low = pandas.read_excel('diet_large.xls', skiprows = 7147, header = 1, nrows = 1)
constraints_low = constraints_low.iloc[:,1:28].fillna(value = 0)
constraints_low.columns = food_data.columns[:27]
constraints_low

,Protein,"Carbohydrate, by difference",Energy,Water,Energy.1,"Calcium, Ca","Iron, Fe","Magnesium, Mg","Phosphorus, P","Potassium, K",...,Vitamin D,"Vitamin C, total ascorbic acid",Thiamin,Riboflavin,Niacin,Pantothenic acid,Vitamin B-6,"Folate, total",Vitamin B-12,Vitamin K (phylloquinone)
0,56,130,2400,3700,2400,1000,8,270,700,4700,...,200,90,0.0012,1.3,16,5,1.3,400,2.4,120


In [6]:
constraints_high = pandas.read_excel('diet_large.xls', skiprows = 7149, header = 1, nrows = 1)
constraints_high = constraints_high.iloc[:,1:28].fillna(value = 0)
constraints_high.columns = food_data.columns[:27]
constraints_high

,Protein,"Carbohydrate, by difference",Energy,Water,Energy.1,"Calcium, Ca","Iron, Fe","Magnesium, Mg","Phosphorus, P","Potassium, K",...,Vitamin D,"Vitamin C, total ascorbic acid",Thiamin,Riboflavin,Niacin,Pantothenic acid,Vitamin B-6,"Folate, total",Vitamin B-12,Vitamin K (phylloquinone)
0,1000000,1000000,1000000,1000000,1000000,2500,45,400,4000,1000000,...,2000,2000,1000000,1000000,35,1000000,100,1000,1000000,1000000


Creating an LP Problem for minimizing the cholesterol content of a meal serving

In [7]:
prob = pulp.LpProblem("diet_cholesterol",LpMinimize)
prob

diet_cholesterol:
MINIMIZE
None
VARIABLES

Getting the cholesterol content of all the food items

In [8]:
cholesterol = dict(zip(food_data.index.tolist(), food_data['Cholesterol'].tolist()))
cholesterol

{'Butter, salted': 215.0,
 'Butter, whipped, with salt': 219.0,
 'Butter oil, anhydrous': 256.0,
 'Cheese, blue': 75.0,
 'Cheese, brick': 94.0,
 'Cheese, brie': 100.0,
 'Cheese, camembert': 72.0,
 'Cheese, caraway': 93.0,
 'Cheese, cheddar': 105.0,
 'Cheese, cheshire': 103.0,
 'Cheese, colby': 95.0,
 'Cheese, cottage, creamed, large or small curd': 15.0,
 'Cheese, cottage, creamed, with fruit': 13.0,
 'Cheese, cottage, nonfat, uncreamed, dry, large or small curd': 7.0,
 'Cheese, cottage, lowfat, 2% milkfat': 8.0,
 'Cheese, cottage, lowfat, 1% milkfat': 4.0,
 'Cheese, cream': 110.0,
 'Cheese, edam': 89.0,
 'Cheese, feta': 89.0,
 'Cheese, fontina': 116.0,
 'Cheese, gjetost': 94.0,
 'Cheese, gouda': 114.0,
 'Cheese, gruyere': 110.0,
 'Cheese, limburger': 90.0,
 'Cheese, monterey': 89.0,
 'Cheese, mozzarella, whole milk': 79.0,
 'Cheese, mozzarella, whole milk, low moisture': 89.0,
 'Cheese, mozzarella, part skim milk': 64.0,
 'Cheese, mozzarella, part skim milk, low moisture': 54.0,
 'Che

Creating a list of all food items

In [9]:
food_items = list(cholesterol.keys())
food_items

['Butter, salted',
 'Butter, whipped, with salt',
 'Butter oil, anhydrous',
 'Cheese, blue',
 'Cheese, brick',
 'Cheese, brie',
 'Cheese, camembert',
 'Cheese, caraway',
 'Cheese, cheddar',
 'Cheese, cheshire',
 'Cheese, colby',
 'Cheese, cottage, creamed, large or small curd',
 'Cheese, cottage, creamed, with fruit',
 'Cheese, cottage, nonfat, uncreamed, dry, large or small curd',
 'Cheese, cottage, lowfat, 2% milkfat',
 'Cheese, cottage, lowfat, 1% milkfat',
 'Cheese, cream',
 'Cheese, edam',
 'Cheese, feta',
 'Cheese, fontina',
 'Cheese, gjetost',
 'Cheese, gouda',
 'Cheese, gruyere',
 'Cheese, limburger',
 'Cheese, monterey',
 'Cheese, mozzarella, whole milk',
 'Cheese, mozzarella, whole milk, low moisture',
 'Cheese, mozzarella, part skim milk',
 'Cheese, mozzarella, part skim milk, low moisture',
 'Cheese, muenster',
 'Cheese, neufchatel',
 'Cheese, parmesan, grated',
 'Cheese, parmesan, hard',
 'Cheese, port de salut',
 'Cheese, provolone',
 'Cheese, ricotta, whole milk',
 'Chee

In [10]:
var = LpVariable.dicts("Food", food_items, lowBound=0, cat='Continuous')   
var

{'Butter, salted': Food_Butter,_salted,
 'Butter, whipped, with salt': Food_Butter,_whipped,_with_salt,
 'Butter oil, anhydrous': Food_Butter_oil,_anhydrous,
 'Cheese, blue': Food_Cheese,_blue,
 'Cheese, brick': Food_Cheese,_brick,
 'Cheese, brie': Food_Cheese,_brie,
 'Cheese, camembert': Food_Cheese,_camembert,
 'Cheese, caraway': Food_Cheese,_caraway,
 'Cheese, cheddar': Food_Cheese,_cheddar,
 'Cheese, cheshire': Food_Cheese,_cheshire,
 'Cheese, colby': Food_Cheese,_colby,
 'Cheese, cottage, creamed, large or small curd': Food_Cheese,_cottage,_creamed,_large_or_small_curd,
 'Cheese, cottage, creamed, with fruit': Food_Cheese,_cottage,_creamed,_with_fruit,
 'Cheese, cottage, nonfat, uncreamed, dry, large or small curd': Food_Cheese,_cottage,_nonfat,_uncreamed,_dry,_large_or_small_curd,
 'Cheese, cottage, lowfat, 2% milkfat': Food_Cheese,_cottage,_lowfat,_2%_milkfat,
 'Cheese, cottage, lowfat, 1% milkfat': Food_Cheese,_cottage,_lowfat,_1%_milkfat,
 'Cheese, cream': Food_Cheese,_cream,


Defining the objective funtion for minimizing the total cholesterol content of the meal serving

$$ minimize: \sum\limits_{i=1}^{n} cholesterol\_content_{i} * serving\_size_{i} \qquad \forall i \in food\_items $$

In [11]:
prob += lpSum([cholesterol[each] * var[each] for each in food_items]), "Total cholesterol in a meal serving"

Defining the constraints:

$$ \sum\limits_{j=1}^{k} nutrient\_content_{ij} * serving\_size_{i} \geqslant minimum\_nutrient\_level \qquad \forall j \in nutrient\_types $$

$$ \sum\limits_{j=1}^{k} nutrient\_content_{ij} * serving\_size_{i} \leqslant maximum\_nutrient\_level \qquad \forall j \in nutrient\_types $$

In [12]:
for i in food_data.columns[0:27].tolist():
    prob += lpSum([food_data[i][each] * var[each] for each in food_items]) >= constraints_low[i]
    prob += lpSum([food_data[i][each] * var[each] for each in food_items]) <= constraints_high[i]

Writing the LP to a file and solving it

In [13]:
prob.writeLP("diet_large.lp")
prob.solve()
print("Status:", LpStatus[prob.status])

Status: Optimal


Getting the values of the optimized solution

In [14]:
for each in prob.variables():
    if (each.varValue > 0):
        print(each.name, "=", each.varValue)

Food_Cereals_ready_to_eat,_GENERAL_MILLS,_HARMONY = 0.011929639
Food_Cereals_ready_to_eat,_QUAKER,_QUAKER_APPLE_ZAPS = 0.0092809948
Food_Infant_formula,_MEAD_JOHNSON,_ENFAMIL,_PROSOBEE_LIPIL,_with_iro = 0.1019902
Food_Infant_formula,_ROSS,_ISOMIL,_with_iron,_powder,_not_reconstitu = 0.92457392
Food_Leavening_agents,_baking_powder,_low_sodium = 0.032337959
Food_Margarine,_regular,_hard,_safflower_and_soybean_(hydrogenated) = 0.22929275
Food_Margarine_like_spread,_reduced_calorie,_40%_fat,_stick,_with_sa = 0.083051744
Food_Nuts,_almond_butter,_plain,_with_salt_added = 0.060746618
Food_Oil,_vegetable_safflower,_salad_or_cooking,_oleic,_over_70%_(pr = 0.03297335
Food_Oil,_whale,_bowhead_(Alaska_Native) = 0.68858764
Food_Sauce,_sofrito,_prepared_from_recipe = 0.37999047
Food_Seeds,_sesame_seed_kernels,_dried_(decorticated) = 0.19395984
Food_Soybeans,_mature_seeds,_dry_roasted = 0.03795924
Food_Soybeans,_mature_seeds,_roasted,_salted = 1.0332571
Food_Spices,_paprika = 0.029252191
Food_Tomato

In [15]:
print("The total cholesterol content of this diet is {}".format(value(prob.objective)))

The total cholesterol content of this diet is 0.0


This diet looks good except for the fact that our solution suggests us to drink abnormally large quantities of water.

This might be due to water containing some essential nutrients with no cholesterol and the optimizer maximizing it for making up essential nutrients and minimizing the cholesterol for our objective.

Moreover, we will not like to suggest the type of water in someone's diet as the person is most likely going to drink the water wherever he/she is staying. As a result, we set the quantity of food_types with water to 0. 



In [16]:

water = ['Water, bottled, PERRIER'
,'Water, bottled, POLAND SPRING'
,'Water, municipal'
,'Water, bottled, non-carbonated, DANNON'
,'Water, bottled, non-carbonated, PEPSI, AQUAFINA'
,'Water, bottled, non-carbonated, DASANI'
,'Water, bottled, non-carbonated, CALISTOGA'
,'Water, bottled, non-carbonated, CRYSTAL GEYSER'
,'Water, bottled, non-carbonated, NAYA'
,'Water, bottled, non-carbonated, DANNON Fluoride To Go'
,'Water, bottled, generic'
,'Water, bottled, non-carbonated, EVIAN'
]

for each in water:
    prob += var[each] <= 0 
    prob += var[each] >= 0

Running the optimizer again


In [17]:
prob.writeLP("diet_large.lp")
prob.solve()
print("Status:", LpStatus[prob.status])

Status: Optimal


Getting the new diet after removing "Water" from food items

In [18]:
for each in prob.variables():
    if (each.varValue > 0):
        print(each.name, "=", each.varValue)

Food_Beans,_adzuki,_mature_seeds,_cooked,_boiled,_without_salt = 0.24772413
Food_Bread_stuffing,_bread,_dry_mix,_prepared = 0.32582028
Food_Caribou,_eye,_raw_(Alaska_Native) = 1.2549083
Food_Cauliflower,_frozen,_cooked,_boiled,_drained,_without_salt = 13.12639
Food_Infant_formula,_ROSS,_ISOMIL,_with_iron,_ready_to_feed = 8.8379628
Food_Milk_substitutes,_fluid,_with_lauric_acid_oil = 0.64545271
Food_Nuts,_coconut_milk,_raw_(liquid_expressed_from_grated_meat_and_ = 1.3392446
Food_Olives,_pickled,_canned_or_bottled,_green = 0.31846651
Food_Onions,_raw = 15.254323
Food_Peas_and_carrots,_frozen,_cooked,_boiled,_drained,_with_salt = 0.71312835


The above diet suggestion makes more sense as we do not have to drink abnormally high quantities of water and fill our bellies

In [19]:
print("The total cholesterol content of this diet is {}".format(value(prob.objective)))

The total cholesterol content of this diet is 0.0
